In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0-rc0


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [7]:
# Parameters for the model and dataset.
TRAINING_SIZE = 1000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789* '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}*{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a * b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS *2 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 1000


In [8]:
questions[0]

'167*155'

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS *2, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS *2)


Vectorization...


In [10]:
len(x[0])

7

In [11]:
len(questions[0])

7

In [12]:
questions[0]

'167*155'

In [13]:
x[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False]])

In [14]:
y[0]

array([[False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [15]:
expected[0]

'25885 '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [17]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(900, 7, 12)
(900, 6, 12)
Validation Data:
(100, 7, 12)
(100, 6, 12)


### Step 3: Create Model

In [18]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [19]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS *2))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Using TensorFlow backend.


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 6, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 6, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.predict(np.array([x_val[0]]))

array([[[0.08211309, 0.04700264, 0.07669315, 0.06843495, 0.10116418,
         0.07610989, 0.09172294, 0.11172233, 0.08369539, 0.08743791,
         0.11252263, 0.06138088],
        [0.068267  , 0.0787926 , 0.07835525, 0.08780963, 0.07802298,
         0.06181432, 0.06398828, 0.09032147, 0.13531153, 0.08363288,
         0.11418976, 0.05949429],
        [0.05545557, 0.05556412, 0.09920757, 0.11854333, 0.06279676,
         0.07161411, 0.06033978, 0.11801219, 0.13258334, 0.08092714,
         0.07517891, 0.06977724],
        [0.05311685, 0.06501643, 0.11789244, 0.12469758, 0.05517573,
         0.06146098, 0.0665104 , 0.11151836, 0.13146251, 0.0695998 ,
         0.07713453, 0.06641441],
        [0.06233798, 0.07909537, 0.10810237, 0.10518776, 0.05564536,
         0.06504069, 0.0553324 , 0.12437715, 0.11674406, 0.08520995,
         0.06026364, 0.08266334],
        [0.04671758, 0.06351384, 0.09388237, 0.10796662, 0.06008872,
         0.08045029, 0.04957479, 0.13898073, 0.09850046, 0.12395296,
  

In [21]:
model.predict_classes(np.array([x_val[0]]))

array([[10,  8,  8,  8,  7,  7]])

### Step 4: Train

In [22]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 1s 667us/step - loss: 2.1420 - acc: 0.3141 - val_loss: 1.8284 - val_acc: 0.3967
Q 155*9   T 1395   ☒       
Q 501*3   T 1503   ☒ 0     
Q 357*50  T 17850  ☒ 496   
Q 815*77  T 62755  ☒ 4000  
Q 736*9   T 6624   ☒       
Q 70*80   T 5600   ☒ 0     
Q 9*14    T 126    ☒       
Q 44*5    T 220    ☒       
Q 8*304   T 2432   ☒       
Q 155*9   T 1395   ☒       

--------------------------------------------------
Iteration 2
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 206us/step - loss: 1.7338 - acc: 0.4161 - val_loss: 1.6811 - val_acc: 0.4217
Q 8*200   T 1600   ☒ 4     
Q 81*429  T 34749  ☒ 5550  
Q 819*547 T 447993 ☒ 28000 
Q 8*372   T 2976   ☒ 4     
Q 113*6   T 678    ☒ 15    
Q 44*5    T 220    ☒ 44    
Q 469*520 T 243880 ☒ 23330 
Q 819*547 T 447993 ☒ 28000 
Q 69*7    

900/900 [==============================] - 0s 274us/step - loss: 1.4069 - acc: 0.5004 - val_loss: 1.4787 - val_acc: 0.4750
Q 466*35  T 16310  ☒ 1382  
Q 1*81    T 81     ☒ 1     
Q 113*6   T 678    ☒ 166   
Q 9*533   T 4797   ☒ 1566  
Q 2*302   T 604    ☒ 168   
Q 53*583  T 30899  ☒ 11500 
Q 9*321   T 2889   ☒ 166   
Q 736*9   T 6624   ☒ 1366  
Q 3*3     T 9      ☒ 1     
Q 9*14    T 126    ☒ 14    

--------------------------------------------------
Iteration 9
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 269us/step - loss: 1.3923 - acc: 0.5037 - val_loss: 1.4655 - val_acc: 0.4900
Q 58*417  T 24186  ☒ 2520  
Q 53*1    T 53     ☒ 10    
Q 384*19  T 7296   ☒ 1222  
Q 10*0    T 0      ☑ 0     
Q 36*88   T 3168   ☒ 1680  
Q 8*52    T 416    ☒ 100   
Q 57*607  T 34599  ☒ 25900 
Q 434*48  T 20832  ☒ 22822 
Q 954*645 T 615330 ☒ 255500
Q 4*4     T 16     ☒ 14    

--------------------------------------------------
Iteration 10
Train on 

900/900 [==============================] - 0s 273us/step - loss: 1.3401 - acc: 0.5283 - val_loss: 1.4389 - val_acc: 0.4933
Q 110*96  T 10560  ☒ 4000  
Q 818*83  T 67894  ☒ 46966 
Q 819*547 T 447993 ☒ 399700
Q 954*645 T 615330 ☒ 355555
Q 4*23    T 92     ☒ 138   
Q 33*4    T 132    ☒ 14    
Q 42*741  T 31122  ☒ 31336 
Q 40*400  T 16000  ☒ 20000 
Q 68*530  T 36040  ☒ 24900 
Q 4*574   T 2296   ☒ 3316  

--------------------------------------------------
Iteration 16
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 273us/step - loss: 1.3250 - acc: 0.5359 - val_loss: 1.4305 - val_acc: 0.4883
Q 91*951  T 86541  ☒ 36736 
Q 871*26  T 22646  ☒ 1996  
Q 6*845   T 5070   ☒ 5400  
Q 29*315  T 9135   ☒ 1116  
Q 4*0     T 0      ☑ 0     
Q 66*3    T 198    ☒ 14    
Q 910*7   T 6370   ☒ 7000  
Q 506*9   T 4554   ☒ 2000  
Q 2*258   T 516    ☒ 111   
Q 36*88   T 3168   ☒ 1446  

--------------------------------------------------
Iteration 17
Train on

In [23]:
x_val[0]

array([[False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False]])

In [24]:
model.predict(np.array([x_val[0]]))

array([[[9.60675720e-03, 1.06874565e-02, 6.15354348e-03, 6.37914836e-02,
         1.29031733e-01, 2.88129896e-01, 1.21408679e-01, 1.21990219e-01,
         7.63375089e-02, 8.30168799e-02, 3.93873788e-02, 5.04584983e-02],
        [1.14652248e-04, 1.04275178e-02, 4.61725704e-02, 1.50854439e-01,
         7.81904832e-02, 1.47715747e-01, 1.15929477e-01, 1.16234317e-01,
         7.93004259e-02, 1.00903064e-01, 4.50590476e-02, 1.09098241e-01],
        [1.92121661e-06, 1.68308150e-03, 3.60821225e-02, 1.95040628e-01,
         4.20177616e-02, 1.87591955e-01, 4.72957119e-02, 1.33522138e-01,
         6.85958937e-02, 9.98696014e-02, 3.51212434e-02, 1.53177902e-01],
        [2.72600187e-06, 8.93714139e-04, 6.10312000e-02, 1.34118468e-01,
         7.06886724e-02, 2.17054233e-01, 6.50985017e-02, 1.11930259e-01,
         8.34267735e-02, 8.12751278e-02, 4.11109291e-02, 1.33369327e-01],
        [3.99843884e-05, 2.28932244e-03, 1.03930712e-01, 8.09781328e-02,
         9.43859443e-02, 2.78943658e-01, 5.1599

In [25]:
model.predict_classes(np.array([x_val[0]]))

array([[5, 3, 3, 5, 5, 5]])